In [10]:
import os
import urllib.request
import numpy as np
import pandas as pd

# Generic ML imports
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import (Lasso, LassoCV, LogisticRegression,
                                  LogisticRegressionCV,LinearRegression,
                                  MultiTaskElasticNet,MultiTaskElasticNetCV)
# EconML imports
from econml.dml import LinearDML, CausalForestDML, SparseLinearDML
from econml.cate_interpreter import SingleTreeCateInterpreter, SingleTreePolicyInterpreter

import matplotlib.pyplot as plt
from networkx.drawing.nx_pydot import to_pydot
from IPython.display import Image,display

import lightgbm as lgb
from sklearn.preprocessing import PolynomialFeatures

import dowhy
from dowhy import CausalModel
from sklearn.preprocessing import StandardScaler
from econml.iv.dr import LinearIntentToTreatDRIV

In [11]:
import pandas as pd
from openpyxl import load_workbook
 
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False,
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.
    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]
    Returns: None
    """
    # from openpyxl import load_workbook
 
    # import pandas as pd
 
    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')
 
    writer = pd.ExcelWriter(filename, engine='openpyxl')
 
    # Python 2.x: define [FileNotFoundError] exception if it doesn't exist
    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError
 
    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)
 
        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row
 
        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)
 
        # copy existing sheets
        writer.sheets = {ws.title: ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass
 
    if startrow is None:
        startrow = 0
 
    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)
 
    # save the workbook
    writer.save()

In [12]:
data=pd.read_excel('excel_output.xls')

In [14]:
data=data.drop(columns=['Unnamed: 0'])

In [15]:
data

,land,square,hunman,factory,school,office,shop,hospital,hotel,theater,...,hospital_Treatment_effect,hotel_Treatment_effect,theater_Treatment_effect,park_Treatment_effect,restaurant_Treatment_effect,edu_Treatment_effect,Health care_Treatment_effect,life_Treatment_effect,scenry_Treatment_effect,transport_Treatment_effect
0,浦东新区,1210.41,4545,5602,716,1828,1639,106,352,5,...,339.634493,203.300646,429.683229,235.210290,215.966837,221.743883,204.892391,230.409640,230.476177,294.266624
1,黄浦区,20.46,32072,125,109,766,378,64,147,14,...,188.294616,201.535072,149.356585,215.821310,207.475040,210.479413,200.223749,204.086703,197.128056,201.157511
2,徐汇区,54.76,19825,519,333,776,293,121,93,3,...,197.636087,200.349696,304.986820,212.867806,209.325046,212.753187,201.100371,211.235589,207.566386,210.013770
3,长宁区,38.30,17982,205,122,613,226,35,103,2,...,134.328118,208.194270,99.637615,204.032822,204.159598,199.733800,208.960804,190.174798,191.563105,189.998693
4,静安区,36.88,28953,455,157,754,400,67,166,8,...,223.357809,200.209643,230.139535,216.844906,208.938340,214.925480,200.343750,211.995330,208.138504,211.060220
5,普陀区,54.83,23387,398,203,518,419,26,71,3,...,182.721598,202.151755,133.702050,184.121758,200.046440,201.850246,204.078421,177.600978,197.003202,157.018840
6,虹口区,23.48,34315,155,117,464,264,29,80,6,...,128.765022,207.947377,51.440695,219.417613,206.940297,207.013148,207.594969,194.119107,195.484542,209.889285
7,杨浦区,60.73,21561,759,388,512,280,46,36,3,...,202.169211,201.956932,294.201111,194.043751,206.493637,207.776997,202.841087,198.072682,205.448199,181.962220
8,闵行区,370.75,6850,3167,433,531,847,61,65,5,...,323.841918,201.725815,392.977772,214.675314,214.352602,219.775966,200.392612,226.437705,225.982217,274.936641
9,宝山区,270.99,7493,1767,212,298,557,30,38,2,...,215.241752,205.797188,323.090190,189.122471,207.637183,208.562482,208.761034,198.688457,212.694375,176.042438


In [16]:
ll=['hunman','factory','school','office','shop','hospital','hotel','theater','park','restaurant','edu','Health care','life','scenry','transport']

In [17]:
year=[2018,2019,2020]

In [18]:
place=['浦东新区',"黄浦区","徐汇区","长宁区","静安区","普陀区","虹口区","杨浦区","闵行区","宝山区","嘉定区","金山区","松江区","青浦区","奉贤区","崇明区",
]

In [19]:
n=0
m=0
for y in year:
    for l in ll:
        data1=data[(data['years']==y)]
        name=l+'_Treatment_effect'
        data_1=data1[["land","years",name]]
        data_1.sort_values(name,inplace=True,ascending=False)
        #print(data_1)
        append_df_to_excel('date-sort-output.xlsx',data_1,sheet_name='Sheet1', startcol=n,startrow=m,index=False)
        n+=4
    m+=20 
    n=0
        


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [20]:
n=0
m=0

for i in range(16):
    data1=data.loc[[0+i,16+i,32+i]]  
    for l in ll:
        
        name=l+'_Treatment_effect'
        data_1=data1[["land","years",name]]
        data_1.sort_values("years",inplace=True,ascending=True)
        #print(data_1)
        append_df_to_excel('years-output.xlsx',data_1,sheet_name='Sheet1', startcol=n,startrow=m,index=False)
        n+=4
    m+=5 
    n=0
        


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
